In [1]:
from sklearn.datasets import *
from sklearn import tree
from dtreeviz.trees import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
regr = tree.DecisionTreeRegressor(max_depth=2)
boston = load_boston()
regr.fit(boston.data, boston.target)

viz = dtreeviz(regr,
               boston.data,
               boston.target,
               target_name='price',
               feature_names=boston.feature_names)

findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


In [3]:
viz.view()

In [4]:
df = pd.read_csv("/home/andronik/repo/decision-tree-learning/data/restaurant.csv")
df.head()

,Alternate,Bar,Fri/Sat,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,WillWait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,No,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No


In [5]:
train, test = train_test_split(df, train_size=0.80, test_size=0.20, random_state=42)

In [6]:
train.shape, test.shape

((9, 11), (3, 11))

In [7]:
train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq
Alternate,9,2,No,5
Bar,9,2,Yes,5
Fri/Sat,9,2,No,5
Hungry,9,2,Yes,5
Raining,9,2,No,5
Reservation,9,2,No,6
WillWait,9,2,Yes,5
Patrons,9,3,Full,5
Price,9,3,$,6
Type,9,4,Burger,4


In [8]:
# The status_group column is the target
target = 'WillWait'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=target)

# Get a list of the numeric features
numeric_features = train.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)

['Alternate', 'Bar', 'Fri/Sat', 'Hungry', 'Patrons', 'Price', 'Raining', 'Reservation', 'Type', 'WaitEstimate']


In [9]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [18]:
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from dtreeviz.trees import *
from sklearn.preprocessing import StandardScaler

In [19]:
pipeline = make_pipeline(
    ce.OneHotEncoder(), 
    SimpleImputer(),
    DecisionTreeClassifier(max_depth=3)
)

In [20]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Alternate', 'Bar', 'Fri/Sat', 'Hungry',
                                     'Patrons', 'Price', 'Raining',
                                     'Reservation', 'Type', 'WaitEstimate'])),
                ('simpleimputer', SimpleImputer()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=3))])

In [21]:
print('Train Accuracy', pipeline.score(X_train, y_train))

Train Accuracy 1.0


In [22]:
y_train

8      No
5     Yes
2     Yes
1      No
11    Yes
4      No
7     Yes
3     Yes
6      No
Name: WillWait, dtype: object

In [23]:
encoder = ce.OneHotEncoder(use_cat_names=True)
imputer = SimpleImputer()
scaler = StandardScaler()

X_train_encoded = encoder.fit_transform(X_train)
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_train_scaled = scaler.fit_transform(X_train_imputed)

In [24]:
X_test_encoded = encoder.transform(X_test)
X_test_imputed = imputer.transform(X_test_encoded)
X_test_scaled = scaler.transform(X_test_imputed)

In [25]:
clas = tree.DecisionTreeClassifier(max_depth=3)  
X_train = X_train_scaled
y_train = [0, 1, 1, 0,1,0,1,1,0]

y_train = np.array(y_train, dtype=np.float32)
clas.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [26]:
from sklearn import metrics

In [31]:
viz = dtreeviz(clas, 
               X_train,
               y_train,
               target_name='WillWait',
               feature_names=['Yes',], 
               class_names=[chr(c) for c in range(ord('0'),ord('9')+1)],
               histtype='bar')

TypeError: dtreeviz() missing 1 required positional argument: 'feature_names'

In [241]:
viz.view()